In [28]:
# Programa main
import os
import yaml
from crewai_tools import PDFSearchTool
from crewai import LLM, Crew, Process
from config import pdf_files, pdf_folder, solicitacoes, controles, restricoes, template
from config import create_agent_leitor, create_agent_revisor, leitor_task, revisor_task
from config import all_articles
import re

# Função para limpar e validar YAML
def clean_and_validate_yaml(yaml_string):
    # Remover caracteres extras ou formatação inesperada
    yaml_string = re.sub(r'```yaml\n|```', '', yaml_string.strip())

    # Adicionar aspas onde necessário (caso contenha caracteres especiais)
    def add_quotes(match):
        return f'{match.group(1)}: "{match.group(2)}"'
    
    yaml_string = re.sub(r'(^\s*\w+):\s*(.+)$', add_quotes, yaml_string, flags=re.MULTILINE)
    
    return yaml_string


for pdf_file_name in pdf_files:

  gpt = LLM(
    model="ollama/mistral",
    base_url="http://localhost:11434"
  )

  pdf = os.path.join(pdf_folder, pdf_file_name)

 # pdf_tool = PDFSearchTool(pdf)

  pdf_tool = PDFSearchTool(pdf,
    config=dict(
        llm=dict(
            provider="ollama", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="phi4",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="ollama", # or openai, ollama, ...
            config=dict(
                model="phi4",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
  )

  agent_leitor = create_agent_leitor(gpt, pdf_tool)
  task_leitor = leitor_task(agent_leitor)

  agent_revisor = create_agent_revisor(gpt)
  task_revisor = revisor_task(agent_revisor)

  crew = Crew(
      agents = [agent_leitor, agent_revisor],
      tasks = [task_leitor, task_revisor],
      process=Process.sequential,
      verbose=True
  )

  inputs = {
      'solicitacoes': solicitacoes,
      'controles': controles,
      'restricoes': restricoes,
      'template': template
  }

  results = crew.kickoff(inputs)

/home/wallace/Projects/leitor de PDF - GPT - Old/myenv/lib/python3.12/site-packages/schema.py:131: ResourceWarning: unclosed <socket.socket fd=77, family=2, type=1, proto=6, laddr=('127.0.0.1', 55524), raddr=('127.0.0.1', 11434)>
  def __init__(
/home/wallace/Projects/leitor de PDF - GPT - Old/myenv/lib/python3.12/site-packages/schema.py:131: ResourceWarning: unclosed <socket.socket fd=78, family=2, type=1, proto=6, laddr=('127.0.0.1', 55526), raddr=('127.0.0.1', 11434)>
  def __init__(
2025-01-10 14:47:36,504 - 139505004748928 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: PDF Reader
## Task: Leia o PDF e responda em YAML às solicitações definidas em <solicitacoes> usando o modelo definido em <template>. 

<solicitacoes>

1 - OBJETIVOS - Identificação dos Objetivos: Realize uma anákise cuidadosa do conteúdo do trabalho para extrair os objetivos principais. Resuma esses objetivos em um paragráfo claro e conciso, capturando a essencia das metas e inteções do estudo.

2 - GAP - Identificação do GAP: Analise o conteúdo do trabalho para identificar o GAP científico que ele aborda, mesmo que não esteja explicitamente mencionado. Formule um paragráfo conciso, focando em destacar a questão central que o estudo procura resolver ou elucidar.

3 - METODOLOGIA - Extração Detalhada da Metodologia: Identificação e Descrição Detalhada da Metodologia. Proceda com uma análise minuciosa para identificar a metodologia utilizada. Detalhe cada aspecto da metodologia utilizada. Detalhe cada aspecto da metodolgia, incluindo o desenho de estudo, as técnicas e ferrament

In [27]:
print(f'Primeiro Resultado: {results} \n')
results = results.raw.replace("```yaml\n", "").replace("\n```", "")
print(f'Segundo resultado {results}')

Primeiro Resultado: artigo: "Framework for Multi-Agent Reinforcement Learning in Robotics"
    OBJETIVOS: "Propose a reinforcement learning from experience (RL) framework for multiple agents in robotics, with a focus on solving multi-controller and multi-task problems."
    GAP: "Challenges exist in the field of RL for multiple agents when it comes to robotics applications. The main issues are the large number of agents needed, as well as the lack of methodologies and tools adequate for solving complex multi-controller and multi-task problems."
    METODOLOGIA: "The framework proposes a hybrid approach, combining a deep reinforcement learning algorithm with declarative programming. The methodology also includes a real-time simulation system and a training and testing mechanism for the agents."
    DATASET: "The dataset used is the Gazebo robotics simulator, as well as movement and sensor data collected from real-robot experiences."
    RESULTADOS: "Results show that the proposed framew

AttributeError: 'str' object has no attribute 'raw'